In [25]:
import rospy
import json
import glob
import rospkg
from os.path import join
from numpy import mean
from scipy.stats import sem
import pylab as plt

In [32]:
def plot_analysis(resfiles, resdir):
        width = 0.2

        fig = plt.figure(facecolor="white")
        ax = fig.add_subplot(111)
        plt.rcParams['font.size'] = 20
        
        labels = ['fixed', 'position only', 'orientation only', 'position \nand orientation']
        colors = ['tomato', 'darkseagreen', 'cornflowerblue', 'slategray']
        conditions = ['fixed', 'orientation_only', 'pose', 'position_only']
        score_dict = {}
        for c in conditions:
            score_dict[c] = []
        
        # plot figure for each result file
        for f in resfiles:
            with open(f) as data_file:
                data = json.load(data_file)
            for c in conditions:
                score_dict[c] += data[c]

        for i in range(len(conditions)):
            score = score_dict[conditions[i]]
            ax.bar((i + 1) / 2.0 - width / 2, mean(score),
                   width, color=colors[i], label=labels[i])
            (line, caps, _) = plt.errorbar((i + 1) / 2.0, mean(score),
                                           sem(score), capsize=3, elinewidth=2, ecolor='k')

            line.remove()
            for cap in caps:
                cap.set_color('k')
                cap.set_markeredgewidth(3)

        legend = ax.legend(loc='upper right')
        # Set the fontsize
        for label in legend.get_texts():
            label.set_fontsize('small')

        for label in legend.get_lines():
            label.set_linewidth(1.5)  # the legend line width

        ax.set_ylabel('Average REBA Score')
        ax.set_ylim([0, 8])
        ax.set_xlim([0, 2.5])

        # save svg file
        savefile = join(resdir, 'average_reba_cost')
        fig.set_size_inches(12.8, 10.24)
        plt.savefig(savefile + '.svg', dpi=100, facecolor=fig.get_facecolor(), transparent=False)
        plt.close()

In [27]:
rospy.init_node('welding_analyzer')

In [28]:
res_folder = 'welding_simulation_gradient'

In [29]:
rospack = rospkg.RosPack()
pkg_expe = rospack.get_path('thr_xp_human_comfort')
directory = join(pkg_expe, 'results', res_folder)

In [30]:
res_files = glob.glob(join(directory, '*', 'reba_cost.json'))

In [33]:
plot_analysis(res_files, directory)

{'pose': [3.311262607574463, 3.311262607574463, 4.37699031829834, 3.385634183883667, 4.644068717956543, 3.090240240097046, 3.8300416469573975, 2.667372941970825, 4.001702308654785, 4.001702785491943, 3.8486194610595703, 3.8486194610595703, 5.67921257019043, 3.8837664127349854, 5.199763774871826, 2.8554961681365967, 3.352242946624756, 2.7676076889038086, 4.384156227111816, 4.384152889251709, 3.081077814102173, 3.081077814102173, 4.3181023597717285, 3.630596399307251, 4.290559768676758, 3.6286864280700684, 4.356265544891357, 3.9867968559265137, 4.340935707092285, 4.340935707092285, 3.4225425720214844, 3.4225425720214844, 2.5637197494506836, 3.570277690887451, 2.4056310653686523, 2.548128128051758, 2.424029588699341, 2.5634207725524902, 2.6301941871643066, 2.6301934719085693, 3.3280463218688965, 3.3280463218688965, 3.5244452953338623, 2.4234390258789062, 3.3011035919189453, 2.7181427478790283, 3.220057249069214, 2.9339632987976074, 2.6629316806793213, 2.6629316806793213, 1.818688392639160